# 구단별 뉴스기사 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import json

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUI 없이 실행 (필요하면 제거)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver를 자동으로 다운로드 및 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 크롤링할 URL
base_url = "https://sports.news.naver.com/kbaseball/news/index"

# 최대 페이지 수 계산
def get_max_page(date, team):
    url = base_url + f"?date={date}&team={team}&page=1&isphoto=N&type=team"
    driver.get(url)
    time.sleep(2)  # 페이지 로딩 대기

    try:
        pagination = driver.find_element(By.CLASS_NAME, "paginate")
        pages = pagination.find_elements(By.TAG_NAME, "a")
        return int(pages[-1].text) if pages else 1
    except:
        return 1

# 기사 목록 크롤링
def crawl_articles(date, team, page):
    articles = []
    url = f"{base_url}?date={date}&team={team}&page={page}&isphoto=N&type=team"
    driver.get(url)
    time.sleep(2)  # 페이지 로딩 대기

    # 뉴스 목록 가져오기
    try:
        news_list = driver.find_elements(By.CSS_SELECTOR, ".news_list > li")
        for item in news_list:
            title_element = item.find_element(By.CSS_SELECTOR, ".title")
            article_url = item.find_element(By.TAG_NAME, "a").get_attribute("href")

            articles.append({
                'news_title': title_element.text.strip(),
                'article_url': article_url
            })
    except:
        pass

    return articles

# 본문 내용 추출
def get_article_content(article_url):
    driver.get(article_url)
    time.sleep(2)  # 페이지 로딩 대기

    try:
        news_content = driver.find_element(By.ID, "newsEndContents").text.strip()
    except:
        news_content = "본문 없음"

    try:
        published_date = driver.find_element(By.CSS_SELECTOR, ".info > span").text.split(' ')[0]
    except:
        published_date = "날짜 없음"

    return {
        'news_content': news_content,
        'published_date': published_date
    }

# 뉴스 기사 크롤링
def crawl_news(date, team):
    max_page = get_max_page(date, team)
    print(f"날짜: {date}, 팀: {team}, 최대 페이지 수: {max_page}")
    
    all_articles = []
    for page in range(1, max_page + 1):
        articles = crawl_articles(date, team, page)
        for article in articles:
            content = get_article_content(article['article_url'])
            all_articles.append({
                'news_title': article['news_title'],
                'news_content': content['news_content'],
                'published_date': content['published_date']
            })
        time.sleep(2)  # 요청 간 2초 대기
    
    return all_articles

def main():
    date = input("크롤링할 날짜를 입력하세요 (YYYYMMDD 형식): ")
    # KIA 타이거즈: HT, 삼성 라이온즈: SS, 두산 베어스: OB, 롯데 자이언츠: LT, KT 위즈: KT, SSG 랜더스: SK, 한화 이글스: HH, NC 다이노스: NC, 키움 히어로즈: WO, LG 트윈스: LG
    team = input("크롤링할 팀 코드를 입력하세요 (예: HT): ")
    
    results = crawl_news(date, team)
    
    with open(f"news_{date}_{team}.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print(f"크롤링 완료. 총 {len(results)}개의 기사를 저장했습니다.")
    driver.quit()

if __name__ == "__main__":
    main()


In [48]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
import pandas as pd

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUI 없이 실행 (필요하면 제거)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver를 자동으로 다운로드 및 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 크롤링할 URL
base_url = "https://sports.news.naver.com/kbaseball/news/index"

# 최대 페이지 수 계산
def get_max_page(date, team):
    url = base_url + f"?date={date}&team={team}&page=1&isphoto=N&type=team"
    driver.get(url)
    time.sleep(2)  # 페이지 로딩 대기

    try:
        pagination = driver.find_element(By.CLASS_NAME, "paginate")
        pages = pagination.find_elements(By.TAG_NAME, "a")
        return int(pages[-1].text) if pages else 1
    except:
        return 1

# 기사 목록 크롤링
def crawl_articles(date, team, max_page):
    articles = []
    
    for page in range(1, max_page + 1):
        url = base_url + f"?date={date}&team={team}&page={page}&isphoto=N&type=team"
        driver.get(url)
        time.sleep(2)  # 페이지 로딩 대기

        # 뉴스 목록 가져오기
        try:
            news_list = driver.find_elements(By.CSS_SELECTOR, "#_newsList > ul > li")
            for item in news_list:
                title_element = item.find_element(By.CSS_SELECTOR, ".title")
                article_url = item.find_element(By.TAG_NAME, "a").get_attribute("href")

                articles.append({
                    'news_title': title_element.text.strip().replace("\\", ""),
                    'article_url': article_url
                })
        except:
            pass

    return articles

# 본문 내용 추출
def get_article_content(article_url):
    driver.get(article_url)
    time.sleep(2)  # 페이지 로딩 대기

    try:
        paragraphs = driver.find_elements(By.CSS_SELECTOR, "span.article_p")
        news_content = " ".join([p.text.strip() for p in paragraphs if p.text.strip()])
    except:
        news_content = "본문 없음"

    return {
        'news_content': news_content
    }

# 뉴스 기사 크롤링
def crawl_news(date, team):
    team_mapping = {
    "HT": "KIA 타이거즈",
    "SS": "삼성 라이온즈",
    "OB": "두산 베어스",
    "LT": "롯데 자이언츠",
    "KT": "KT 위즈",
    "SK": "SSG 랜더스",
    "HH": "한화 이글스",
    "NC": "NC 다이노스",
    "WO": "키움 히어로즈",
    "LG": "LG 트윈스"
    }

    published_date = date[:4] + "-" + date[4:6] + "-" + date[6:]
    team_name = team_mapping.get(team, team)

    max_page = get_max_page(date, team)
    print(f"날짜: {published_date}, 팀: {team_name}, 최대 페이지 수: {max_page}")
    
    all_articles = []
    articles = crawl_articles(date, team, max_page)
    for article in articles:
        content = get_article_content(article['article_url'])
        all_articles.append({
            'news_title': article['news_title'],
            'news_content': content['news_content'],
            'published_date': published_date
        })
    time.sleep(2)  # 요청 간 2초 대기
    
    return all_articles

# CSV 저장 함수
def save_to_csv(data, filename="news_results.csv"):
    df = pd.DataFrame(data)
    df.to_csv(f"news_csv/{filename}", index=False, encoding="utf-8-sig")  # 한글 깨짐 방지 위해 utf-8-sig 사용
    print(f"CSV 파일 저장 완료: {filename}")

def main():
    date = input("크롤링할 날짜를 입력하세요 (YYYYMMDD 형식): ")
    # KIA 타이거즈: HT, 삼성 라이온즈: SS, 두산 베어스: OB, 롯데 자이언츠: LT, KT 위즈: KT, SSG 랜더스: SK, 한화 이글스: HH, NC 다이노스: NC, 키움 히어로즈: WO, LG 트윈스: LG
    team = input("크롤링할 팀 코드를 입력하세요 (예: HT): ")
    
    results = crawl_news(date, team)
    
    with open(f"news_json/news_{date}_{team}.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    save_to_csv(results, f"news_{date}_{team}.csv")
    
    print(f"크롤링 완료. 총 {len(results)}개의 기사를 저장했습니다.")
    driver.quit()

if __name__ == "__main__":
    main()


날짜: 2025-03-13, 팀: KIA 타이거즈, 최대 페이지 수: 4
CSV 파일 저장 완료: news_20250313_HT.csv
크롤링 완료. 총 73개의 기사를 저장했습니다.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import re

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUI 없이 실행 (필요하면 제거)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver를 자동으로 다운로드 및 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 크롤링할 URL
article_url = 'https://m.sports.naver.com/kbaseball/article/109/0005260906'

driver.get(article_url)
time.sleep(2)  # 페이지 로딩 대기

try:
    paragraphs = driver.find_elements(By.CSS_SELECTOR, "span.article_p")
    news_content = " ".join([p.text.strip() for p in paragraphs if p.text.strip()])
except:
    news_content = "본문 없음"

print(news_content)

[OSEN=부산, 이상학 기자] “빠른 공에만 너무 포커스가 맞춰지는데…”
13일 부산 사직구장에서 열린 프로야구 시범경기 롯데-한화전. 김경문 한화 감독은 지난 11일 문학 SSG전에서 최고 시속 159.7km 강속구를 뿌리며 큰 화제가 된 문동주에 대한 질문이 나오자 웃음꽃을 피우며 “(어깨 통증 이후 첫 등판이라) 기대하지 말라 그랬는데 좋았다. 팔 스윙이 작년 좋았을 때보다 좋게 나왔다. 내가 본 것 중에선 거의 베스트였다”고 칭찬했다.
문동주에 이어 나온 김서현도 최고 시속 156km를 뿌리며 한화팬들을 밥 안 먹어도 배부르게 했다. 최고 154km까지 던진 신인 정우주까지 ‘젊은 파이어볼러 삼총사’가 구축된 한화는 외국인 투수 코디 폰세, 라이언 와이스 그리고 불펜 한승혁, 박상원 등 150km를 던지는 강속구 투수들이 넘친다.
‘강속구 군단’ 한화에 관심이 높아지고 있는 가운데 김경문 감독은 “(관심이) 고맙기도 하고, 공 빠른 것이 자랑거리가 될 수 있지만 너무 거기에 포커스가 맞춰지지 않았으면 좋겠다. 야구는 공 빠른 것 외에 정교한 제구가 필요하다. (권)민규 같은 어린 친구도 볼이 빨라서 잘 던지는 게 아니다. 야구는 요소 요소에 밀고 넣는 강약 조절과 제구력이 굉장히 중요하다”고 말했다.
가르친다고 되는 게 아닌 강속구는 축복받은 재능이고, 최근 몇 년 사이 고교 최고 강속구 투수들을 꾸준히 모은 한화는 타팀의 부러움을 한몸에 받고 있다. 하지만 김 감독은 강속구 풍요 속에 제구력의 중요성을 강조했고, 19세 신인 권민규를 예로 들었다. 권민규 이야기가 나올 때마다 김 감독은 “스카우들이 잘 뽑았다”고 말하는데 13일 롯데전에서 그 이유를 확인할 수 있었다.
선발 류현진에 이어 5회 구원등판한 권민규는 첫 타자 전민재에게 초구 직구, 3구째 슬라이더를 모두 존에 넣어 투스트라이크를 잡았다. 4~5구 연속 파울이 됐지만 6구째 몸쪽 낮은 슬라이더로 헛스윙 삼진 처리했다.
이어 좌타자 황성빈 상대로도 1~2구 연속 슬라이더, 직구를 바깥쪽에 

# 경기별 뉴스기사 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
import pandas as pd

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUI 없이 실행 (필요하면 제거)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver를 자동으로 다운로드 및 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 크롤링할 URL
base_url = "https://m.sports.naver.com"
# https://m.sports.naver.com/kbaseball/schedule/index?date=2025-03-16
# https://m.sports.naver.com/game/20250316HHNC02025/news

# 일자별 경기 페이지 목록
def get_game_list(date):
    date_form = date[:4] + "-" + date[4:6] + "-" + date[6:]
    url = base_url + f"/kbaseball/schedule/index?date={date_form}"
    driver.get(url)
    time.sleep(2)  # 페이지 로딩 대기

    games = driver.find_elements(By.CSS_SELECTOR, "#content > div > div:nth-child(4) > div:nth-child(1) > ul > li > div > div.MatchBox_match_area__39dEr > a")
    return games

# 기사 목록 크롤링
def crawl_articles(date, team, max_page):
    articles = []
    
    for page in range(1, max_page + 1):
        url = base_url + f"?date={date}&team={team}&page={page}&isphoto=N&type=team"
        driver.get(url)
        time.sleep(2)  # 페이지 로딩 대기

        # 뉴스 목록 가져오기
        try:
            news_list = driver.find_elements(By.CSS_SELECTOR, "#_newsList > ul > li")
            for item in news_list:
                title_element = item.find_element(By.CSS_SELECTOR, ".title")
                article_url = item.find_element(By.TAG_NAME, "a").get_attribute("href")

                articles.append({
                    'news_title': title_element.text.strip().replace("\\", ""),
                    'article_url': article_url
                })
        except:
            pass

    return articles

# 본문 내용 추출
def get_article_content(article_url):
    driver.get(article_url)
    time.sleep(2)  # 페이지 로딩 대기

    try:
        paragraphs = driver.find_elements(By.CSS_SELECTOR, "span.article_p")
        news_content = " ".join([p.text.strip() for p in paragraphs if p.text.strip()])
    except:
        news_content = "본문 없음"

    return {
        'news_content': news_content
    }

# 뉴스 기사 크롤링
def crawl_news(date, team):
    team_mapping = {
    "HT": "KIA 타이거즈",
    "SS": "삼성 라이온즈",
    "OB": "두산 베어스",
    "LT": "롯데 자이언츠",
    "KT": "KT 위즈",
    "SK": "SSG 랜더스",
    "HH": "한화 이글스",
    "NC": "NC 다이노스",
    "WO": "키움 히어로즈",
    "LG": "LG 트윈스"
    }

    published_date = date[:4] + "-" + date[4:6] + "-" + date[6:]
    team_name = team_mapping.get(team, team)

    max_page = get_max_page(date, team)
    print(f"날짜: {published_date}, 팀: {team_name}, 최대 페이지 수: {max_page}")
    
    all_articles = []
    articles = crawl_articles(date, team, max_page)
    for article in articles:
        content = get_article_content(article['article_url'])
        all_articles.append({
            'news_title': article['news_title'],
            'news_content': content['news_content'],
            'published_date': published_date
        })
    time.sleep(2)  # 요청 간 2초 대기
    
    return all_articles

# CSV 저장 함수
def save_to_csv(data, filename="news_results.csv"):
    df = pd.DataFrame(data)
    df.to_csv(f"news_csv/{filename}", index=False, encoding="utf-8-sig")  # 한글 깨짐 방지 위해 utf-8-sig 사용
    print(f"CSV 파일 저장 완료: {filename}")

def main():
    date = input("크롤링할 날짜를 입력하세요 (YYYYMMDD 형식): ")
    # KIA 타이거즈: HT, 삼성 라이온즈: SS, 두산 베어스: OB, 롯데 자이언츠: LT, KT 위즈: KT, SSG 랜더스: SK, 한화 이글스: HH, NC 다이노스: NC, 키움 히어로즈: WO, LG 트윈스: LG
    team = input("크롤링할 팀 코드를 입력하세요 (예: HT): ")
    
    results = crawl_news(date, team)
    
    with open(f"news_json/news_{date}_{team}.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    save_to_csv(results, f"news_{date}_{team}.csv")
    
    print(f"크롤링 완료. 총 {len(results)}개의 기사를 저장했습니다.")
    driver.quit()

if __name__ == "__main__":
    main()


KeyboardInterrupt: Interrupted by user

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
import pandas as pd

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUI 없이 실행 (필요하면 제거)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver를 자동으로 다운로드 및 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 크롤링할 URL
base_url = "https://m.sports.naver.com"

# 일자별 경기 페이지 목록
def get_game_list(date):
    date_form = date[:4] + "-" + date[4:6] + "-" + date[6:]
    url = base_url + f"/kbaseball/schedule/index?date={date_form}"
    driver.get(url)
    time.sleep(2)  # 페이지 로딩 대기

    games = driver.find_elements(By.CSS_SELECTOR, "#content > div > div:nth-child(4) > div:nth-child(1) > ul > li > div > div.MatchBox_match_area__39dEr > a")

    game_list = []
    for game in games:
        game_url = game.get_attribute("href")
        home_team = game.find_element(By.CSS_SELECTOR, "div.MatchBox_team__1YR1v:nth-child(1) > div.MatchBox_team_name__2k3YB").text
        away_team = game.find_element(By.CSS_SELECTOR, "div.MatchBox_team__1YR1v:nth-child(2) > div.MatchBox_team_name__2k3YB").text
        game_list.append({
            'game_url': game_url,
            'home_team': home_team,
            'away_team': away_team
        })
    return game_list

# 기사 목록 크롤링
def crawl_articles(date, team):
    articles = []
    
    for page in range(1, max_page + 1):
        url = base_url + f"?date={date}&team={team}&page={page}&isphoto=N&type=team"
        driver.get(url)
        time.sleep(2)  # 페이지 로딩 대기

        # 뉴스 목록 가져오기
        try:
            news_list = driver.find_elements(By.CSS_SELECTOR, "#_newsList > ul > li")
            for item in news_list:
                title_element = item.find_element(By.CSS_SELECTOR, ".title")
                article_url = item.find_element(By.TAG_NAME, "a").get_attribute("href")

                articles.append({
                    'news_title': title_element.text.strip().replace("\\", ""),
                    'article_url': article_url
                })
        except:
            pass

    return articles

date="20240731"
game_list=get_game_list(date)
print(game_list)

['https://m.sports.naver.com/game/20240731HHKT02024', 'https://m.sports.naver.com/game/20240731LTSK02024', 'https://m.sports.naver.com/game/20240731NCWO02024', 'https://m.sports.naver.com/game/20240731OBHT02024', 'https://m.sports.naver.com/game/20240731SSLG02024']


# 뉴스기사를 활용한 요약 Test

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Choose your prompt
prompt = "Explain how wonderful you are"  # English example
prompt = "스스로를 자랑해 봐"       # Korean example

messages = [
    {"role": "system", 
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128,
    do_sample=False,
)
print(tokenizer.decode(output[0]))

c:\Users\SSAFY\Desktop\Specialized_Project\S12P21B206\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\SSAFY\Desktop\Specialized_Project\S12P21B206\venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SSAFY\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or

In [1]:
import json
from datetime import datetime
from transformers import pipeline

# 뉴스 데이터 로드 함수
def load_news_data(date, team_code):
    filename = f"news_json/{date}/news_{date}_{team_code}.json"
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

# 뉴스 요약 함수
def summarize_news(text):
    summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base")
    summary = summarizer(text, max_length=50, min_length=30, do_sample=False)[0]['summary_text']
    return summary

# 뉴스 하이라이트 생성 함수
def generate_highlights(news_data):
    highlights = []
    for news in news_data:
        title = news['news_title']
        content = news['news_content']
        summary = summarize_news(content)
        highlights.append(f"{title}: {summary}")
        if len(highlights) >= 5:
            break
    return highlights

# 메인 함수
def main():
    date = "20250322"
    team_code = "LG"  # 예시로 LG 팀 코드 사용
    
    news_data = load_news_data(date, team_code)
    highlights = generate_highlights(news_data)
    
    print(f"{date} {team_code} 뉴스 하이라이트")
    for i, highlight in enumerate(highlights, 1):
        print(f"{i}. {highlight}")

if __name__ == "__main__":
    main()

c:\Users\SSAFY\Desktop\Specialized_Project\S12P21B206\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\SSAFY\Desktop\Specialized_Project\S12P21B206\venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SSAFY\.cache\huggingface\hub\models--t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administr

20250322 LG 뉴스 하이라이트
1. '9년의 간절함.' 팬들까지 DM 넣어 부활시킨 응원가가 있다. '포에버 LG'에 문보경도 "소름 돋았다"[잠실 현장]: ''  '' LG'' , '2023', 2025'''"" . ""  "", ", " 
2. "1호 홈런 아닐까했는데, 기분 좋아"…'타구속도 169.8km' 대포! 25년 1호 아치의 주인공의 미소 [MD잠실]: LG   22 2025  SOL Bank KBO 1-0 112 .  3 128.2km - , 169.8km
3. 9년 만에 잠실로 돌아온 'Forever LG' 선수·팬 모두 감동…"팬들 열기 상상 이상, 소름 돋았다" [잠실 현장]: 'Forever LG'  '2025' 2 'forever lg'  2025 'soon to be released' . 'Song from a secret garden'
4. ‘1회 3점→3회 4점→12:2 대승’ 타선 대폭발…염갈량 "1회 분위기 가져와, 치리노스의 KBO 첫 승 진심 축하” [오!쎈 잠실]: osen.co.kr is a global leader in energy efficiency . the company is the world's largest energy supplier . it is the only company in the world to offer energy efficiency products .
5. 타점왕 대신 4번, 염갈량 구상 적중…"좀 더 믿을 수 있는 선수 되겠다" LG 문보물, 홈런포로 존재감 과시 [잠실 현장]: 'Forever LG' is a tv series starring lgbt's samsung galaxy s5 . ''  , ''','


In [ ]:
date = "20250323"

target_date = datetime.date(int(date[:4]), int(date[4:6]), int(date[6:]))

print(target_date)

TypeError: 'int' object is not subscriptable